In [ ]:
DIR='/home/RootFiles/'

import ROOT
%jsroot on

canvas = ROOT.TCanvas("MainCanvas")

# Some usefull functions

In [ ]:
def get_histogram(file_dir, key, **hist_args):
    with ROOT.TFile.Open(f"{DIR}/{file_dir}") as file:
        hist = file.demo.Get(key)
        hist.SetDirectory(0)
        
    if "title" in hist_args:
        title = hist_args["title"]
        hist.SetTitle(title)
    
    if "scale" in hist_args:
        scale = hist_args["scale"]
        hist.Scale(scale)
        
    return hist
        
def histogram_stack(name, historams, **stack_args):
    stack = ROOT.THStack(name , "")
    for hist in historams:
        stack.Add(hist)
    return stack


# Take steps to produce histograms...

<div dir ="rtl">
    
<p>الان می‌خواهیم ببینیم که رویدادهای با 4 لپتون از کجا می‌آیند.<br>بنابراین لازم است برخوردهای پروتون-پروتون را شبیه‌سازی کنیم.<br>بیایید به رویدادهای شبیه‌سازی‌شده pp-&gt;ZZ-&gt;4muons نگاه کنیم. </p>
 
</div> 

In [ ]:
hzz4mu12_invmass = get_histogram("2012/simulation/ZZ4mu12.root", "mass4mu_8TeV_low")
hzz4mu12_invmass.Draw("HIST")
canvas.Draw()

<div dir ="rtl">
    
<p>اما این نمودار به خودی خود معنا ندارد.  برای اینکه تعبیر درستی از آن داشته باشیم، باید بدانیم چه تعداد رویداد برای کشیدن این نمودار استفاده شده‌اند.<br>برای سمپل ZZ4mu12 تعداد 1499064 رویداد تولید شده است. اگر تعداد در هر بین هیستوگرام به این مقدار کل تقسیم شود، پس می‌توان نمودار را به شکل احتمال ترجمه کرد. </p>
</div> 

In [ ]:
nevtZZ4mu12 = 1499064
hzz4mu12_invmass = get_histogram("2012/simulation/ZZ4mu12.root", "mass4mu_8TeV_low", scale=1.0/nevtZZ4mu12)
hzz4mu12_invmass.Draw("Hist")
canvas.Draw()

<div dir ="rtl">
    
<p>از این نمودار دریافت می‌شود که چه کسری از رویدادهای pp-&gt;ZZ-&gt;4mu در سال 2012، 4 میون دارند و جرم ناوردا این 4 میون به چه صورتی توزیع شده است.  حالا سوال این است که اساسا چه تعداد از این رویدادها انتظار می‌رود در سال 2012 تولید شده باشند؟<br>برای پاسخ به این سوال، لازم است کمیت درخشندگی (luminosity) کل را برای سال 2012 بدانیم. اگر پارامتر درخشندگی را در سطح‌مقطع تولید این فرایند در انرژی مرکز جرم 8 تراالکترون‌ولت ضرب کنیم، تعداد انتظاری ما از این فرایند به‌دست می‌آید. </p>
</div>    

In [ ]:
xsecZZ412 = 0.077
lumi12 = 11580.
sfZZ = 1.386

totalNumberExpectedZZ = xsecZZ412 * lumi12 * sfZZ

hzz4mu12_invmass = get_histogram("2012/simulation/ZZ4mu12.root", "mass4mu_8TeV_low", scale=1.0/nevtZZ4mu12)

hzz4mu12_invmass.Scale(totalNumberExpectedZZ)
hzz4mu12_invmass.Draw('HIST')
canvas.Draw()

<div dir="rtl">
    
<p>حالا این نمودار به درستی نماینده رویدادهای pp-&gt;ZZ-&gt;4mu در سال 2012 در این آنالیز است.<br>ما می‌توانیم این تمرین را برای هر دو سال 2011 و 2012 و حالت‌های مختلف 4 الکترون، 4 میون و 2الکترون-2میون تکرار کنیم.</p>
</div>

# Histogram of $ZZ \rightarrow 4l$ Events

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)

lumi = {12:11580. , 11:2330.}

sfZZ = 1.386

zz_xsections = {12:{'4e':0.077 , '4mu':0.077 , '2mu2e':0.18},
                11:{'4e':0.067 , '4mu':0.067 , '2mu2e':0.15}}

zz_totals = {12:{'4mu':1499064 , '4e':1499093 , '2mu2e':1497445},
            11:{'4mu':1447136 , '4e':1493308 , '2mu2e':1479879}}

hists = []

# Decay possibility 
for decaymode in ['4mu' , '4e' , '2mu2e']:
    
    # For the two years
    for year in [11,12]:
        
        # Choose wich file do you want to open
        file_directory  = f"20{year}/simulation/ZZ{decaymode}{year}.root"
        
        # ... and which histogram do you want to read
        file_key        = f"mass{decaymode}_8TeV_low"
        
        # ... also, with a scale factor and a title
        scale_factor    = zz_xsections[year][decaymode]*lumi[year]*sfZZ / zz_totals[year][decaymode] 
        histogram_title = f"{decaymode.replace('mu' , '#mu')}, 20{year}"
        
        # ... then, read the histogram
        h = get_histogram(file  = file_directory,
                          key   = file_key,
                          scale = scale_factor,
                          title = histogram_title)
        
        # ... finally append to the `hists` array.
        hists.append(h)

# Draw!
ZZ4LepPlot = histogram_stack("ZZ4LeptPlot", hists)
ZZ4LepPlot.Draw("pfc plc pmc HIST")
ZZ4LepPlot.GetXaxis().SetTitle('m_{4l} [GeV]')

canvas.BuildLegend(0.7, 0.7, 0.95, 0.95 , "Invariant of 4 leptons", "f")
canvas.Draw()

<div dir="rtl">

<p>این نمودار می‌تواند برای رویدادهای TTbar و DY نیز تولید شود. برای این منظور به اطلاعات سطح‌مقطع و تعداد کل این رویدادها نیز نیاز داریم. سعی کنید این نمودارها را تولید کنید:</p>
</div> 

# Histogram of $t\bar{t}$ Events 

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)

sfTTBar = {11:0.11 , 12:1 }
tt_xsections = {12:200. , 11:177.31}
tt_totals = {12:6423106 , 11:9771205}

hists = []

for year in [11,12]:
    file_directory = f"20{year}/simulation/TTBar{year}.root"
    for decaymode in ['4mu' , '4e' , '2mu2e']:
        
        file_key        = f"mass{decaymode}_8TeV_low"
        scale_factor    = tt_xsections[year]*lumi[year]*sfTTBar[year] / tt_totals[year]
        histogram_title = "{0}, 20{1}".format( decaymode.replace('mu' , '#mu') ,year)
        
        # TODO:
        current_hist = ...
        ...
        ...
        ...
        
        
        
TTPlot = histogram_stack("TTPlot",hists)

TTPlot.Draw("pfc plc pmc HIST")
TTPlot.GetXaxis().SetTitle('m_{4l} [GeV]')

canvas.BuildLegend(0.7, 0.7, 0.95, 0.95 , "Invariant of 4 leptons", "f")
canvas.Draw()

# Histogram of Drell-Yan Events

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)

sfDY = 1.12
dy_xsections = {12:{10:10.742 , 50:2955.} ,
               11:{10:9507. , 50:2475.}}
dy_totals = {12:{10:6462290 , 50:29426492},
             11:{10:39909640, 50:36408225}}

hists = []
for mass in [10,50]:
    fname_extension = 'TuneZ' if mass == 50 else ''
    for year in [11,12]:
        file_directory = f"20{year}/simulation/DY{mass}{fname_extension}{year}.root"
        
        for decaymode in ['4mu' , '4e' , '2mu2e']:
            file_key = f"mass{decaymode}_8TeV_low"
            scale_factor = dy_xsections[year][mass]*lumi[year]*sfDY / dy_totals[year][mass]
            histogram_title = "DY({2}), {0}, 20{1}".format( decaymode.replace('mu' , '#mu') ,year , mass)
            
            # TODO:
            current_hist = ...
            ...
            ...
            ...

DYPlot = histogram_stack("DYtPlot" , hists)

DYPlot.Draw("pfc plc pmc HIST")


DYPlot.GetXaxis().SetTitle('m_{4l} [GeV]')

canvas.BuildLegend(0.7, 0.7, 0.95, 0.95 , "Invariant of 4 leptons", "f")
canvas.Draw()